In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import torch.optim as optim

# import numpy as np
import pandas as pd
import ast
import math
import time
import random

from models.rnn.encoder_decoder_gru import EncoderRNN, DecoderRNN
from models.rnn.combined_networks import train
from rnn_utils import tensorsFromPair
from utils import showPlot, timeSince, asMinutes
from lang import load_data

%load_ext autoreload
%autoreload 2

# Training Loops

In [2]:
input_seq = torch.Tensor([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]) # -> binary encoded integer sequence
target_seq = torch.Tensor([0, 2, 5, 6]) # -> character encoded function

training_pairs = [[input_seq, target_seq]]

In [3]:
eq, seq, pairs = load_data()

In [4]:
print(pairs[:2])
print(pairs[0][0])
print(pairs[0][1])

[('24,28,32,36,40,44,48,52', '4*t+4+2*8'), ('36,37,38,39,40,41,42,43', '0+5*8-5+t')]
24,28,32,36,40,44,48,52
4*t+4+2*8


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
from utils import eq_to_seq
def converted_loss(decoder_output, target):
    print(decoder_output)
    print(target)
    
    decoded_words = []
    # output_lang.index2word[topi.item()]
    topv, topi = decoder_output.data.topk(1)
    
    print(eq.index2word[topi.item()])



    # output = eq_to_seq(self.decoded_representation, len(self.param["target_sequence"])) 

    # fitness: float = 0.0
    
    # for i, value in enumerate(target_sequence):
    #     fitness -= min(1000, (value - output[i])**2)

    # fitness /= len(target_sequence)

    #fitness += 0.0 # eg.: closeness to actual target value
    #fitness -= 0.0 # eg.: number of symbols

    return 0



In [7]:
def training(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs), seq, eq)
                      for i in range(n_iters)]
    criterion =nn.NLLLoss() #  converted_loss

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [30]:
hidden_size = 256
encoder = EncoderRNN(seq.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, eq.n_words).to(device)

training(encoder, decoder, 50000, print_every=100)

0m 15s (- 132m 58s) (100 0%) 2.1639
0m 20s (- 85m 49s) (200 0%) 1.7180
0m 25s (- 70m 39s) (300 0%) 1.6271
0m 30s (- 62m 4s) (400 0%) 1.5830
0m 34s (- 56m 38s) (500 1%) 1.6162
0m 38s (- 53m 2s) (600 1%) 1.6321
0m 42s (- 50m 16s) (700 1%) 1.6138
0m 46s (- 47m 48s) (800 1%) 1.6033
0m 50s (- 46m 3s) (900 1%) 1.5938
0m 54s (- 44m 48s) (1000 2%) 1.6080
0m 59s (- 43m 47s) (1100 2%) 1.5745
1m 2s (- 42m 36s) (1200 2%) 1.5847
1m 6s (- 41m 41s) (1300 2%) 1.5592
1m 11s (- 41m 22s) (1400 2%) 1.5360
1m 19s (- 42m 58s) (1500 3%) 1.5201
1m 24s (- 42m 40s) (1600 3%) 1.5515
1m 28s (- 41m 58s) (1700 3%) 1.5224
1m 32s (- 41m 15s) (1800 3%) 1.5144
1m 36s (- 40m 39s) (1900 3%) 1.5547
1m 40s (- 40m 6s) (2000 4%) 1.5098
1m 45s (- 40m 0s) (2100 4%) 1.5050
1m 52s (- 40m 51s) (2200 4%) 1.5203
1m 57s (- 40m 42s) (2300 4%) 1.5047
2m 1s (- 40m 17s) (2400 4%) 1.5273
2m 8s (- 40m 43s) (2500 5%) 1.4977
2m 14s (- 40m 44s) (2600 5%) 1.4976
2m 18s (- 40m 26s) (2700 5%) 1.5360
2m 22s (- 40m 0s) (2800 5%) 1.5028
2m 26s (- 

KeyboardInterrupt: 

In [34]:
from rnn_utils import tensorFromSentence

MAX_LENGTH = 10
input_lang = seq
output_lang = eq
EOS_token = 0
SOS_token = 1

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            # decoder_output, decoder_hidden, decoder_attention = decoder(
            #     decoder_input, decoder_hidden, encoder_outputs)
            # decoder_attentions[di] = decoder_attention.data

            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)  # this if or simply decoder
                
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [35]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [36]:
evaluateRandomly(encoder, decoder)

> 4,5,6,7,8,9,10,11
= 3-2*6*0+t
< t + t - t + 1 * 1 <EOS>

> -2,-7,-12,-17,-22,-27,-32,-37
= t+2+1-t*6
< t - t * t * t * t <EOS>

> -6,-5,-4,-3,-2,-1,0,1
= 1-5-8+t+5
< t - t - t - t - t <EOS>

> 11,13,15,17,19,21,23,25
= t*2+9-t+t
< t + t + t + t + t <EOS>

> -1,0,1,2,3,4,5,6
= t+8+4-7-7
< t - t - t - t - t <EOS>

> 6,7,8,9,10,11,12,13
= 7+0+6+t-8
< t + 1 - t + 1 * 1 <EOS>

> 16,20,24,28,32,36,40,44
= 9+5+t*4-2
< t + 4 * 4 + 4 + t <EOS>

> 3,12,27,48,75,108,147,192
= t*3*t+5*0
< t * t * t * t * t <EOS>

> 0,-7,-14,-21,-28,-35,-42,-49
= 2-t-t*6+5
< 7 - t * t * t * t <EOS>

> 4,8,12,16,20,24,28,32
= t*3+0*7+t
< t * t - t * t - t <EOS>

